In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree

# Dados NSL-KDD

In [2]:
kddtrain = pd.read_csv(r"C:\Users\luang\OneDrive\Documentos\Mestrado\Códigos Python\Autoencoder-based Network Anomaly Detection\KDDTrain.csv")
kddtest = pd.read_csv(r"C:\Users\luang\OneDrive\Documentos\Mestrado\Códigos Python\Autoencoder-based Network Anomaly Detection\KDDTest.csv")

In [3]:
x = pd.concat((kddtrain,kddtest), axis=0)
x = pd.get_dummies(x)
data_class = x['class_anomaly']
data = x.drop(['class_anomaly', 'class_normal'], axis = 1) 

In [4]:
data = data.to_numpy()
data_class = data_class.to_numpy()

# Abordagem Autoencoder-Algoritmos

In [5]:
epochs = 100
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    
    #####################################################
    ####Pré-Processamento
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 0.50
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_normal = np.array(list(set(np.arange(0, noTrain)) - set(numbers_knn_normal)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = normalTrain[numbers_autoencoder_normal]
    autoencoderTrain_class = np.zeros(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####Autoencoder
    #Modela as camadas do Autoencoder
    var = autoencoderTrain.shape[1]
    input_vector = keras.Input(shape=(var,))
    en1 = layers.Dense(32, activation=activation)(input_vector)
    en2 = layers.Dense(16, activation=activation)(en1)
    de1 = layers.Dense(32, activation=activation)(en2)
    de2 = layers.Dense(var, activation=activation)(de1)

    #Gera o modelo
    autoencoder = keras.Model(input_vector, de2)
    
    #Gera modelo do Encoder
    encoder = keras.Model(input_vector, en2)
    
    #Cria Decoder
    encoded_input = keras.Input(shape=(16,))
    dec_layer = autoencoder.layers[-2](encoded_input)
    decoder_layer = autoencoder.layers[-1](dec_layer)
    decoder = keras.Model(encoded_input, decoder_layer)

    #Compila o modelo
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    #Treina o modelo
    history = autoencoder.fit(autoencoderTrain, autoencoderTrain, epochs = epochs, batch_size = batch_size, shuffle = True)
    
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    pred_knn = autoencoder.predict(knnTrain)
    mse_knn = np.mean(np.power(knnTrain - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                 'true_class': knnTrain_class})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_knn[:,0].reshape(-1,1), axis=1)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(error_encoder, error_knn[:,1])
    
    #Teste do Modelo
    pred_knn = autoencoder.predict(modelTest)
    mse_knn = np.mean(np.power(modelTest - pred_knn, 2), axis=1)
    error_df_knn = pd.DataFrame({'reconstruction_error': mse_knn,
                                'true_class': modelTest_class})
    error_knn = error_df_knn.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer, error_knn[:,0].reshape(-1,1), axis=1)
    predict_knn = neigh.predict(error_encoder)
    
    #Métricas
    accuracy_knn[count] = accuracy_score(error_knn[:,1], predict_knn[:])
    recall_knn[count] = recall_score(error_knn[:,1], predict_knn[:])
    precision_knn[count] = precision_score(error_knn[:,1], predict_knn[:])
    f1_knn[count] = f1_score(error_knn[:,1], predict_knn[:])
    matrix_knn[count] = confusion_matrix(error_knn[:,1], predict_knn[:])
     
    #####################################################
    ####Decisition Tree
    ###Treinamento Decisition Tree 
    pred_dt = autoencoder.predict(knnTrain)
    mse_dt = np.mean(np.power(knnTrain - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': knnTrain_class})
    error_dt = error_df_dt.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_dt[:,0].reshape(-1,1), axis=1)
    dt = tree.DecisionTreeClassifier()
    dt.fit(error_encoder, error_dt[:,1])

    #Teste
    pred_dt = autoencoder.predict(modelTest)
    mse_dt = np.mean(np.power(modelTest - pred_dt, 2), axis=1)
    error_df_dt = pd.DataFrame({'reconstruction_error': mse_dt,
                                     'true_class': modelTest_class})
    error_dt = error_df_dt.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_dt[:,0].reshape(-1,1), axis=1)
    predict_dt = dt.predict(error_encoder)

    #Métricas
    accuracy_dt[count] = accuracy_score(error_dt[:,1], predict_dt[:])
    recall_dt[count] = recall_score(error_dt[:,1], predict_dt[:])
    precision_dt[count] = precision_score(error_dt[:,1], predict_dt[:])
    f1_dt[count] = f1_score(error_dt[:,1], predict_dt[:])
    matrix_dt[count] = confusion_matrix(error_dt[:,1], predict_dt[:])
    
    #####################################################
    ####SVM
    #Treinamento SVM
    pred_svm = autoencoder.predict(knnTrain)
    mse_svm = np.mean(np.power(knnTrain - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': knnTrain_class})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(knnTrain)
    error_encoder = np.append(encoder_layer, error_svm[:,0].reshape(-1,1), axis=1)
    clf = svm.SVC(kernel='rbf')
    clf.fit(error_encoder, error_svm[:,1])
    
    #Teste
    pred_svm = autoencoder.predict(modelTest)
    mse_svm = np.mean(np.power(modelTest - pred_svm, 2), axis=1)
    error_df_svm = pd.DataFrame({'reconstruction_error': mse_svm,
                                     'true_class': modelTest_class})
    error_svm = error_df_svm.to_numpy()
    encoder_layer = encoder.predict(modelTest)
    error_encoder = np.append(encoder_layer,error_svm[:,0].reshape(-1,1), axis=1)
    predict_svm = clf.predict(error_encoder)
    
    #Métricas
    accuracy_svm[count] = accuracy_score(error_svm[:,1], predict_svm[:])
    recall_svm[count] = recall_score(error_svm[:,1], predict_svm[:])
    precision_svm[count] = precision_score(error_svm[:,1], predict_svm[:])
    f1_svm[count] = f1_score(error_svm[:,1], predict_svm[:])
    matrix_svm[count] = confusion_matrix(error_svm[:,1], predict_svm[:])
    
    count = count + 1
    fold_no = fold_no + 1# Abordagem Autoencoder-Algoritmos

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 2ms/step - loss: 0.0541
Epoch 2/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0387
Epoch 3/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0221
Epoch 4/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0189
Epoch 5/100
136/136 [=============================

136/136 [==============================] - 0s 3ms/step - loss: 8.5690e-04
Epoch 87/100
136/136 [==============================] - 0s 2ms/step - loss: 8.5460e-04
Epoch 88/100
136/136 [==============================] - 0s 2ms/step - loss: 8.5245e-04
Epoch 89/100
136/136 [==============================] - 0s 2ms/step - loss: 8.5023e-04
Epoch 90/100
136/136 [==============================] - 0s 2ms/step - loss: 8.4829e-04
Epoch 91/100
136/136 [==============================] - 0s 2ms/step - loss: 8.4636e-04
Epoch 92/100
136/136 [==============================] - 0s 2ms/step - loss: 8.4434e-04
Epoch 93/100
136/136 [==============================] - 0s 2ms/step - loss: 8.4225e-04
Epoch 94/100
136/136 [==============================] - 0s 2ms/step - loss: 8.4059e-04
Epoch 95/100
136/136 [==============================] - 0s 2ms/step - loss: 8.3869e-04
Epoch 96/100
136/136 [==============================] - 0s 3ms/step - loss: 8.3697e-04
Epoch 97/100
136/136 [==============================] - 

136/136 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 51/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 52/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 53/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 54/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 55/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 56/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 57/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 58/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031A: 0s - loss: 0.003
Epoch 59/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 60/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 61/100
136/136 [==============================] - 0s 2ms/step - loss: 0.003

136/136 [==============================] - 0s 2ms/step - loss: 0.0039
Epoch 16/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 17/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 18/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 19/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 20/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 21/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 22/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0037
Epoch 23/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 24/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 25/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0036A: 0s - loss:
Epoch 26/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0036
Epoc

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
136/136 [==============================] - 1s 2ms/step 

...

136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 78/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 79/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 80/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 81/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078A: 0s - loss: 0
Epoch 82/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 83/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078A: 0s - loss: 0.00
Epoch 84/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 85/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 86/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 87/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 88/100
136/136 [==============================] - 0s 3ms/step

136/136 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 42/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 43/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 44/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 45/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 46/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0025
Epoch 47/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 48/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 49/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 50/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 51/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 52/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 53/100
136/

136/136 [==============================] - 0s 3ms/step - loss: 0.0039
Epoch 7/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0037
Epoch 8/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0034
Epoch 9/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 10/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 11/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0030
Epoch 12/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0029
Epoch 13/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 14/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 15/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 16/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 17/100
136/136 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 18/100
136/136

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/100
Please report this to the TensorFlow team. When filing 

136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 72/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 73/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 74/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 75/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 76/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 77/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 78/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 79/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 80/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 81/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 82/100
136/136 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 83/100
136/

In [6]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[7598.8  106.6]
 [ 121.5 7024.8]] 
Acurácia:  0.9846414984954326  +/-  0.0009518079919519159 
Recall:  0.9829982427712217  +/-  0.0011518763256459818 
Precision:  0.9850537514845271  +/-  0.0014045894597178377 
F1:  0.9840242381489139  +/-  0.0009871288019598192

Autoencoder - SVM:
 [[7523.5  181.9]
 [ 301.2 6845.1]] 
Acurácia:  0.9674717618295954  +/-  0.001755472332532105 
Recall:  0.9578523471220166  +/-  0.00264692948852867 
Precision:  0.974127939027414  +/-  0.0033325844110567693 
F1:  0.965915698558025  +/-  0.001821205555893994

Autoencoder - Decision Tree:
 [[7574.4  131. ]
 [ 110.8 7035.5]] 
Acurácia:  0.9837190402309044  +/-  0.0009648096279486039 
Recall:  0.9844954605869596  +/-  0.0013823380777123531 
Precision:  0.9817227405890983  +/-  0.0014750500376360639 
F1:  0.9831060854182052  +/-  0.0009999434702205195


In [7]:
Result = np.array(["KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_IL+RE.txt', Result, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [8]:
Result_Matrix = np.array(["KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_IL+RE_Matrix.txt', Result_Matrix, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
